In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import os

print(tf.__version__)
print(keras.__version__)

1.13.1
2.2.4-tf


In [2]:
tf.enable_eager_execution()

In [3]:
learning_rate = 0.001
training_epochs = 50
batch_size = 1
img_size = 224
n_train = 2000
lr_decay_ratio = 0.05
lr_decay_epoch_num = 10
alpha = tf.constant(0.2)

tf.set_random_seed(777)

In [4]:
tfrecord_train = 'facenet_train.tfrecord'
tfrecord_dir = 'tfrecords'

In [5]:
train_tfr_path = os.path.join(os.getcwd(), tfrecord_dir, tfrecord_train)
print(os.path.exists(train_tfr_path))

True


In [6]:
def _parse_function(tfrecord_serialized):
    features={'anc_image' : tf.FixedLenFeature([], tf.string),
             'pos_image' : tf.FixedLenFeature([], tf.string),
             'neg_image_bundle' : tf.FixedLenFeature([], tf.string)}
    parsed_features = tf.parse_single_example(tfrecord_serialized, features)
    
    anc_image = tf.decode_raw(parsed_features['anc_image'], tf.uint8)
    anc_image = tf.reshape(anc_image, [img_size, img_size, 3])
    anc_image = tf.cast(anc_image, tf.float32)/255.
    
    pos_image = tf.decode_raw(parsed_features['pos_image'], tf.uint8)
    pos_image = tf.reshape(pos_image, [img_size, img_size, 3])
    pos_image = tf.cast(pos_image, tf.float32)/255.
    
    neg_image_bundle = tf.decode_raw(parsed_features['neg_image_bundle'], tf.uint8)
    neg_image_bundle = tf.reshape(neg_image_bundle, [15, img_size, img_size, 3])
    neg_image_bundle = tf.cast(neg_image_bundle, tf.float32)/255.
    
    return anc_image, pos_image, neg_image_bundle
    
    

In [7]:
train_dataset = tf.data.TFRecordDataset(train_tfr_path)
train_dataset = train_dataset.map(_parse_function, num_parallel_calls=8)
train_dataset = train_dataset.shuffle(buffer_size=n_train*2).prefetch(buffer_size=batch_size).batch(batch_size)

In [8]:
from tensorflow.keras.layers import Input, Conv2D, Dense, ReLU, BatchNormalization, MaxPool2D, GlobalAveragePooling2D, Concatenate, Lambda

In [9]:
def create_model():
    inputs = Input(shape=(img_size, img_size, 3))
    net = Conv2D(64, 7, 2, 'SAME')(inputs)
    net = BatchNormalization()(net)
    net = ReLU()(net)
    net = MaxPool2D(pool_size=3, strides=2, padding='SAME')(net)
    net = Conv2D(64, 1, 1, 'SAME')(net)
    net = Conv2D(192, 3, 1, 'SAME')(net)
    net = BatchNormalization()(net)
    net = ReLU()(net)
    net = MaxPool2D(3, 2, 'SAME')(net)
    #####3a
    a_a = Conv2D(64, 1, 1, 'SAME')(net)
    a_b = Conv2D(96, 1, 1, 'SAME')(net)
    a_b = Conv2D(128, 3, 1, 'SAME')(a_b)
    a_c = Conv2D(16, 1, 1, 'SAME')(net)
    a_c = Conv2D(32, 5, 1, 'SAME')(a_c)
    a_d = MaxPool2D(3, 1, 'SAME')(net)
    a_d = Conv2D(32, 1, 1, 'SAME')(a_d)
    net = Concatenate()([a_a, a_b, a_c, a_d])
    net = BatchNormalization()(net)
    net = ReLU()(net)

    ######3b
    a_a = Conv2D(64, 1, 1, 'SAME')(net)
    a_b = Conv2D(96, 1, 1, 'SAME')(net)
    a_b = Conv2D(128, 3, 1, 'SAME')(a_b)
    a_c = Conv2D(32, 1, 1, 'SAME')(net)
    a_c = Conv2D(64, 5, 1, 'SAME')(a_c)
    #a_d = Lambda(lambda x: l2_norm_pooling(x))(net)
    a_d = MaxPool2D(3, 1, 'SAME')(net)
    a_d = Conv2D(64, 1, 1, 'SAME')(a_d)
    net = Concatenate()([a_a, a_b, a_c, a_d])
    
    ###### 3c
    a_b = Conv2D(128, 1, 1, 'SAME')(net)
    a_b = Conv2D(256, 3, 2, 'SAME')(a_b)
    a_c = Conv2D(32, 1, 1, 'SAME')(net)
    a_c = Conv2D(64, 5, 2, 'SAME')(a_c)
    a_d = MaxPool2D(3, 2, 'SAME')(net)
    a_d = Conv2D(320, 1, 1, 'SAME')(a_d)
    net = Concatenate()([a_b, a_c, a_d])
    
    ###### 4a
    a_a = Conv2D(256, 1, 1, 'SAME')(net)
    a_b = Conv2D(96, 1, 1, 'SAME')(net)
    a_b = Conv2D(192, 3, 1, 'SAME')(a_b)
    a_c = Conv2D(32, 1, 1, 'SAME')(net)
    a_c = Conv2D(64, 5, 1, 'SAME')(a_c)
    #a_d = Lambda(lambda x: l2_norm_pooling(x))(net)
    a_d = MaxPool2D(3, 1, 'SAME')(net)
    a_d = Conv2D(128, 1, 1, 'SAME')(a_d)
    net = Concatenate()([a_a, a_b, a_c, a_d])
    
    ###### 4b
    a_a = Conv2D(224, 1, 1, 'SAME')(net)
    a_b = Conv2D(112, 1, 1, 'SAME')(net)
    a_b = Conv2D(224, 3, 1, 'SAME')(a_b)
    a_c = Conv2D(32, 1, 1, 'SAME')(net)
    a_c = Conv2D(64, 5, 1, 'SAME')(a_c)
    #a_d = Lambda(lambda x: l2_norm_pooling(x))(net)
    a_d = MaxPool2D(3, 1, 'SAME')(net)
    a_d = Conv2D(128, 1, 1, 'SAME')(a_d)
    net = Concatenate()([a_a, a_b, a_c, a_d])
    
    ###### 4c
    a_a = Conv2D(192, 1, 1, 'SAME')(net)
    a_b = Conv2D(128, 1, 1, 'SAME')(net)
    a_b = Conv2D(256, 3, 1, 'SAME')(a_b)
    a_c = Conv2D(32, 1, 1, 'SAME')(net)
    a_c = Conv2D(64, 5, 1, 'SAME')(a_c)
    #a_d = Lambda(lambda x: l2_norm_pooling(x))(net)
    a_d = MaxPool2D(3, 1, 'SAME')(net)
    a_d = Conv2D(128, 1, 1, 'SAME')(a_d)
    net = Concatenate()([a_a, a_b, a_c, a_d])
    
    ###### 4d
    a_a = Conv2D(160, 1, 1, 'SAME')(net)
    a_b = Conv2D(144, 1, 1, 'SAME')(net)
    a_b = Conv2D(288, 3, 1, 'SAME')(a_b)
    a_c = Conv2D(32, 1, 1, 'SAME')(net)
    a_c = Conv2D(64, 5, 1, 'SAME')(a_c)
    #a_d = Lambda(lambda x: l2_norm_pooling(x))(net)
    a_d = MaxPool2D(3, 1, 'SAME')(net)
    a_d = Conv2D(128, 1, 1, 'SAME')(a_d)
    net = Concatenate()([a_a, a_b, a_c, a_d])
    
    ###### 4e    
    a_b = Conv2D(160, 1, 1, 'SAME')(net)
    a_b = Conv2D(256, 3, 2, 'SAME')(a_b)
    a_c = Conv2D(64, 1, 1, 'SAME')(net)
    a_c = Conv2D(128, 5, 2, 'SAME')(a_c)
    #a_d = Lambda(lambda x: l2_norm_pooling(x))(net)
    a_d = MaxPool2D(3, 2, 'SAME')(net)
    a_d = Conv2D(640, 1, 1, 'SAME')(a_d)
    net = Concatenate()([a_b, a_c, a_d])
    
    ###### 5a
    a_a = Conv2D(384, 1, 1, 'SAME')(net)
    a_b = Conv2D(192, 1, 1, 'SAME')(net)
    a_b = Conv2D(384, 3, 1, 'SAME')(a_b)
    a_c = Conv2D(48, 1, 1, 'SAME')(net)
    a_c = Conv2D(128, 5, 1, 'SAME')(a_c)
    #a_d = Lambda(lambda x: l2_norm_pooling(x))(net)
    a_d = MaxPool2D(3, 1, 'SAME')(net)
    a_d = Conv2D(128, 1, 1, 'SAME')(a_d)
    net = Concatenate()([a_a, a_b, a_c, a_d])
    
    ###### 5b
    a_a = Conv2D(384, 1, 1, 'SAME')(net)
    a_b = Conv2D(192, 1, 1, 'SAME')(net)
    a_b = Conv2D(384, 3, 1, 'SAME')(a_b)
    a_c = Conv2D(48, 1, 1, 'SAME')(net)
    a_c = Conv2D(128, 5, 1, 'SAME')(a_c)
    #a_d = Lambda(lambda x: l2_norm_pooling(x))(net)
    a_d = MaxPool2D(3, 1, 'SAME')(net)
    a_d = Conv2D(128, 1, 1, 'SAME')(a_d)
    net = Concatenate()([a_a, a_b, a_c, a_d])
    
    ###### avg pool and fully conn
    net = GlobalAveragePooling2D()(net)
    net = Dense(units=128)(net)
    

    return keras.Model(inputs=inputs, outputs=net)

In [10]:
model = create_model()
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_v1 (BatchNo (None, 112, 112, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 112, 112, 64) 0           batch_normalization_v1[0][0]     
_____________________________________

In [11]:
def loss_fn(model, anc_image, pos_image, neg_image_bundle):
    logit_anc = model(anc_image, training=True)
    logit_pos = model(pos_image, training=True)
    logit_anc = tf.div(logit_anc, tf.math.sqrt(tf.reduce_sum(tf.square(logit_anc), 1)))
    #print(logit_anc)
    logit_pos = tf.div(logit_pos, tf.math.sqrt(tf.reduce_sum(tf.square(logit_pos), 1)))
    neg_image_bundle = tf.reshape(neg_image_bundle, (15, img_size, img_size, 3))
    logit_neg = model(neg_image_bundle, training=True)
    
    neg_list = []
    for i in range(len(logit_neg)):
        neg_list.append(tf.div(logit_neg[i], tf.math.sqrt(tf.reduce_sum(tf.square(logit_neg[i]), 0))))
    
    logit_neg = tf.stack(neg_list)
    
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(logit_anc, logit_pos)), 1)
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(logit_anc, logit_neg)), 1)    
    
    temp_sub = tf.subtract(pos_dist, neg_dist)
    temp_min = -tf.reduce_min(tf.math.abs(temp_sub))
    
    #print(tf.add(temp_min, alpha))
    
    
    return tf.add(temp_min, alpha)
    

In [12]:
def grad(model, anc_image, pos_image, neg_image_bundle):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, anc_image, pos_image, neg_image_bundle)
    return tape.gradient(loss, model.variables)

In [13]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

In [14]:
for epoch in range(training_epochs):
    avg_loss = 0
    train_step = 0
    
    for anc_image, pos_image, neg_image_bundle in train_dataset:        
        grads = grad(model, anc_image, pos_image, neg_image_bundle)
        optimizer.apply_gradients(zip(grads, model.variables))
        loss = loss_fn(model, anc_image, pos_image, neg_image_bundle)
        avg_loss = avg_loss + loss
        train_step += 1
    avg_loss = avg_loss / train_step
    
    print('Epoch:', '{}'.format(epoch + 1), 'loss =', '{:.8f}'.format(avg_loss))

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


KeyboardInterrupt: 